**PRE-ANALYSIS**

First of all, we will start exploring our dataset in order to familiarize with the different rows and columns. After this analysis, we will be able to notice whether it is needed or not to clean and transform the available data. Moreover, we will come up with interesting questions that me be answered after analysing it deeply.

In [90]:
#Libraries importation

import pandas as pd #data transformation and manipulation.
import datetime
pd.set_option("display.max_columns", None) #to display all columns of the dataframe.

In [91]:
# Let's connect to the file patent_raw.txt

df_pat = pd.read_csv("../1_Data/Data_Raw/patent_raw.txt", delimiter="~") #we checked the delimiter by opening the txt.
df_pat.head()

,Appl_Type,Appl_No,Product_No,Patent_No,Patent_Expire_Date_Text,Drug_Substance_Flag,Drug_Product_Flag,Patent_Use_Code,Delist_Flag,Submission_Date
0,N,20610,1,7625884,"Aug 24, 2026",NaN,NaN,U-141,NaN,NaN
1,N,18613,1,7560445,"Feb 1, 2027",Y,Y,U-986,NaN,NaN
2,N,19734,2,8455524,"Apr 18, 2027",NaN,NaN,U-1029,NaN,"Jun 27, 2013"
3,N,19734,5,8455524,"Apr 18, 2027",NaN,NaN,U-1029,NaN,NaN
4,N,20832,7,7422388,"Apr 25, 2027",NaN,Y,U-1397,NaN,NaN


In [92]:
df_pat.tail()

,Appl_Type,Appl_No,Product_No,Patent_No,Patent_Expire_Date_Text,Drug_Substance_Flag,Drug_Product_Flag,Patent_Use_Code,Delist_Flag,Submission_Date
20047,N,210874,4,6515117*PED,"Apr 4, 2026",NaN,NaN,NaN,NaN,NaN
20048,N,210874,4,7919598*PED,"Jun 16, 2030",NaN,NaN,NaN,NaN,NaN
20049,N,210874,4,8716251*PED,"Sep 21, 2028",NaN,NaN,NaN,NaN,NaN
20050,N,210874,4,8501698*PED,"Dec 20, 2027",NaN,NaN,NaN,NaN,NaN
20051,N,210874,4,9616028*PED,"May 12, 2031",NaN,NaN,NaN,NaN,NaN


In [93]:
#Lets retrieve more information about this txt file we have just charged (only exclusivities).

print(f"The file has {df_pat.shape[0]} rows and {df_pat.shape[1]} columns.")
df_pat.info()

The file has 20052 rows and 10 columns.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Appl_Type                20052 non-null  object
 1   Appl_No                  20052 non-null  int64 
 2   Product_No               20052 non-null  int64 
 3   Patent_No                20052 non-null  object
 4   Patent_Expire_Date_Text  20052 non-null  object
 5   Drug_Substance_Flag      3341 non-null   object
 6   Drug_Product_Flag        9538 non-null   object
 7   Patent_Use_Code          11578 non-null  object
 8   Delist_Flag              78 non-null     object
 9   Submission_Date          17044 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.5+ MB


**Notes**
- Need to drop columns Drug_Substance_Flag and Drug_Product_Flag, not interesting for the analysis.
- Column names are standarized (first letter of each word in uppercase and spaces are substituted by underscore).
- From columns 0 to 4 there are no null values. We should manage null values in columns from 5 to 9.
- Patent Use Codes are not descriptive. Maybe interesting to map patent use codes with patent descriptions in file patent_use_codes_raw.xlsx
- Regarding data types, the columns Patent_Expire_Date_Text and Submission_Date should be converted into datetime.
- We should check for duplicates before and after data cleaning and transformation.
- Check for unique values in text columns. Maybe we need specific transformations.

In [94]:
df_pat.drop(['Drug_Substance_Flag', 'Drug_Product_Flag' ], axis = 1, inplace = True) #drop unnecessary columns.

In [95]:
#Check for duplicated rows.

print(f"There are {df_pat.duplicated().sum()} duplicates in this dataframe.")

There are 0 duplicates in this dataframe.


In [96]:
#Now let's check for the unique values of the columns
pd.set_option("display.max_rows",None) #to display maximum possible rows.
display(df_pat['Appl_Type'].unique()) #To know which are the unique values for application types.
display(df_pat['Delist_Flag'].unique()) #To know which are the unique values for delist flag.



array(['N'], dtype=object)

array([nan, 'Y'], dtype=object)

- Just one option for the application Type (Appl_Type) column: N. It would be interesting to substitute it by a more recognisable label: Innovator.
- Two options for Delist_Flag: Y and NaN. Y will be substituted by "Requested" and null values (NaN) will be filled with "Not Requested".

In [97]:
#We substitute N by Innovator as it stands for NDA (New Drug Application), A by Generic as it stands for ANDA (Abbreviated New Drug Application). Finally every different value or nulls stay the same.
df_pat['Appl_Type'] = df_pat['Appl_Type'].apply(lambda x: 'Innovator' if x=='N' else 'Generic' if x=='A' else 'NaN' if pd.isnull(x) else x)

In [98]:
#Delist_Flag column transformations

df_pat['Delist_Flag'] = df_pat['Delist_Flag'].apply(lambda x: 'Requested' if x=='Y' else 'Not Requested' if pd.isnull(x) else x)

In [99]:
display(df_pat.head())

,Appl_Type,Appl_No,Product_No,Patent_No,Patent_Expire_Date_Text,Patent_Use_Code,Delist_Flag,Submission_Date
0,Innovator,20610,1,7625884,"Aug 24, 2026",U-141,Not Requested,NaN
1,Innovator,18613,1,7560445,"Feb 1, 2027",U-986,Not Requested,NaN
2,Innovator,19734,2,8455524,"Apr 18, 2027",U-1029,Not Requested,"Jun 27, 2013"
3,Innovator,19734,5,8455524,"Apr 18, 2027",U-1029,Not Requested,NaN
4,Innovator,20832,7,7422388,"Apr 25, 2027",U-1397,Not Requested,NaN


In [100]:
display(df_pat['Appl_Type'].value_counts())
display(df_pat['Delist_Flag'].value_counts())

Appl_Type
Innovator    20052
Name: count, dtype: int64

Delist_Flag
Not Requested    19974
Requested           78
Name: count, dtype: int64

In [101]:
#Fill null values for Submission_Date
df_pat['Submission_Date'].fillna("Unknown", inplace=True)

C:\Users\juanj\AppData\Local\Temp\ipykernel_3896\1255707175.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_pat['Submission_Date'].fillna("Unknown", inplace=True)


In [102]:
df_pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Appl_Type                20052 non-null  object
 1   Appl_No                  20052 non-null  int64 
 2   Product_No               20052 non-null  int64 
 3   Patent_No                20052 non-null  object
 4   Patent_Expire_Date_Text  20052 non-null  object
 5   Patent_Use_Code          11578 non-null  object
 6   Delist_Flag              20052 non-null  object
 7   Submission_Date          20052 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.2+ MB


In [103]:
#Now I wil rename the column Patent_Expire_Date_Text

df_pat.rename(columns={'Patent_Expire_Date_Text':'Expiration_Date'}, inplace=True)

In [104]:
#Now let's convert both date columns into datetime. First we need to perform some transformations...

month_dict = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12
}

def mapping_month(x):
    for key, value in month_dict.items():
        if key in x:
            return x.replace(key, str(value))
    return x  # Return the original value if no match is found

df_pat[['Expiration_Date','Submission_Date']] = df_pat[['Expiration_Date','Submission_Date']].map(mapping_month) #map applies the function to both columns in one code line.
df_pat.head()

,Appl_Type,Appl_No,Product_No,Patent_No,Expiration_Date,Patent_Use_Code,Delist_Flag,Submission_Date
0,Innovator,20610,1,7625884,"8 24, 2026",U-141,Not Requested,Unknown
1,Innovator,18613,1,7560445,"2 1, 2027",U-986,Not Requested,Unknown
2,Innovator,19734,2,8455524,"4 18, 2027",U-1029,Not Requested,"6 27, 2013"
3,Innovator,19734,5,8455524,"4 18, 2027",U-1029,Not Requested,Unknown
4,Innovator,20832,7,7422388,"4 25, 2027",U-1397,Not Requested,Unknown


In [105]:
#To convert date columns into datetime first we need to apply some transformations...

df_pat[['Expiration_Date','Submission_Date']] = df_pat[['Expiration_Date','Submission_Date']].apply(lambda x: x.str.replace(", ","/").str.replace(" ","/"))
df_pat.head()

,Appl_Type,Appl_No,Product_No,Patent_No,Expiration_Date,Patent_Use_Code,Delist_Flag,Submission_Date
0,Innovator,20610,1,7625884,8/24/2026,U-141,Not Requested,Unknown
1,Innovator,18613,1,7560445,2/1/2027,U-986,Not Requested,Unknown
2,Innovator,19734,2,8455524,4/18/2027,U-1029,Not Requested,6/27/2013
3,Innovator,19734,5,8455524,4/18/2027,U-1029,Not Requested,Unknown
4,Innovator,20832,7,7422388,4/25/2027,U-1397,Not Requested,Unknown


In [106]:
#Conversion into datetime

df_pat[['Expiration_Date','Submission_Date']]  = df_pat[['Expiration_Date','Submission_Date']].apply(lambda col: pd.to_datetime(col, errors='coerce')) #apply function is useful to apply a funtion to multiple columns of a dataframe.
df_pat.head()

C:\Users\juanj\AppData\Local\Temp\ipykernel_3896\2300016559.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pat[['Expiration_Date','Submission_Date']]  = df_pat[['Expiration_Date','Submission_Date']].apply(lambda col: pd.to_datetime(col, errors='coerce')) #apply function is useful to apply a funtion to multiple columns of a dataframe.


,Appl_Type,Appl_No,Product_No,Patent_No,Expiration_Date,Patent_Use_Code,Delist_Flag,Submission_Date
0,Innovator,20610,1,7625884,2026-08-24,U-141,Not Requested,NaT
1,Innovator,18613,1,7560445,2027-02-01,U-986,Not Requested,NaT
2,Innovator,19734,2,8455524,2027-04-18,U-1029,Not Requested,2013-06-27
3,Innovator,19734,5,8455524,2027-04-18,U-1029,Not Requested,NaT
4,Innovator,20832,7,7422388,2027-04-25,U-1397,Not Requested,NaT


In [107]:
#Finally we will create a new column called Patent_Duration

df_pat['Patent_Duration_Years'] = round((df_pat['Expiration_Date'] -  df_pat['Submission_Date']).dt.days / 365.25,1) #to years

df_pat.head()

,Appl_Type,Appl_No,Product_No,Patent_No,Expiration_Date,Patent_Use_Code,Delist_Flag,Submission_Date,Patent_Duration_Years
0,Innovator,20610,1,7625884,2026-08-24,U-141,Not Requested,NaT,NaN
1,Innovator,18613,1,7560445,2027-02-01,U-986,Not Requested,NaT,NaN
2,Innovator,19734,2,8455524,2027-04-18,U-1029,Not Requested,2013-06-27,13.8
3,Innovator,19734,5,8455524,2027-04-18,U-1029,Not Requested,NaT,NaN
4,Innovator,20832,7,7422388,2027-04-25,U-1397,Not Requested,NaT,NaN


In [108]:
df_pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Appl_Type              20052 non-null  object        
 1   Appl_No                20052 non-null  int64         
 2   Product_No             20052 non-null  int64         
 3   Patent_No              20052 non-null  object        
 4   Expiration_Date        20052 non-null  datetime64[ns]
 5   Patent_Use_Code        11578 non-null  object        
 6   Delist_Flag            20052 non-null  object        
 7   Submission_Date        17044 non-null  datetime64[ns]
 8   Patent_Duration_Years  17044 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 1.4+ MB


In [109]:
#Lets turn Appl_No and Product_No into objects as they are not going to be used for calculations.
df_pat['Appl_No'] = df_pat['Appl_No'].astype(str)
df_pat['Product_No'] = df_pat['Product_No'].astype(str)

In [110]:
df_pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Appl_Type              20052 non-null  object        
 1   Appl_No                20052 non-null  object        
 2   Product_No             20052 non-null  object        
 3   Patent_No              20052 non-null  object        
 4   Expiration_Date        20052 non-null  datetime64[ns]
 5   Patent_Use_Code        11578 non-null  object        
 6   Delist_Flag            20052 non-null  object        
 7   Submission_Date        17044 non-null  datetime64[ns]
 8   Patent_Duration_Years  17044 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 1.4+ MB


In [111]:
df_pat['Patent_Use_Code'].unique()

array(['U-141', 'U-986', 'U-1029', ..., 'U-2347', nan, 'U-2719'],
      dtype=object)

In [112]:
df_pat['Patent_Use_Code'].fillna("No Code", inplace=True) #fill null values with Unknown.

C:\Users\juanj\AppData\Local\Temp\ipykernel_3896\2202292629.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_pat['Patent_Use_Code'].fillna("No Code", inplace=True) #fill null values with Unknown.


In [113]:
#Lets turn the file containing patent use codes and descriptions (patent_use_codes_raw.xlsx) into a dataframe.
df_patent_description = pd.read_excel("../1_Data/Data_Raw/patent_use_codes_raw.xlsx", sheet_name="Sheet1") #we checked the delimiter by opening the txt.
df_patent_description.head()

,Code,Definition
0,U-1,PREVENTION OF PREGNANCY
1,U-2,TREATMENT OR PROPHYLAXIS OF ANGINA PECTORIS AN...
2,U-3,TREATMENT OF HYPERTENSION
3,U-4,PROVIDING PREVENTION AND TREATMENT OF EMESIS A...
4,U-5,METHOD OF PRODUCING BRONCHODILATION


In [114]:
df_pat['Patent_Use'] = df_pat['Patent_Use_Code'].map(dict(zip(df_patent_description['Code'], df_patent_description['Definition']))).fillna("No Code")#map applies the function to both columns in one code line.
df_pat.head()

,Appl_Type,Appl_No,Product_No,Patent_No,Expiration_Date,Patent_Use_Code,Delist_Flag,Submission_Date,Patent_Duration_Years,Patent_Use
0,Innovator,20610,1,7625884,2026-08-24,U-141,Not Requested,NaT,NaN,TREATMENT OF ULCERATIVE COLITIS
1,Innovator,18613,1,7560445,2027-02-01,U-986,Not Requested,NaT,NaN,TREATMENT OF PATIENTS INFECTED WITH PEDICULUS ...
2,Innovator,19734,2,8455524,2027-04-18,U-1029,Not Requested,2013-06-27,13.8,METHOD FOR TREATING ACUTE ELEVATIONS OF BLOOD ...
3,Innovator,19734,5,8455524,2027-04-18,U-1029,Not Requested,NaT,NaN,METHOD FOR TREATING ACUTE ELEVATIONS OF BLOOD ...
4,Innovator,20832,7,7422388,2027-04-25,U-1397,Not Requested,NaT,NaN,USE AS AN ANTISEPTIC FOR THE PREPARATION OF A ...


In [115]:
df_pat.columns

Index(['Appl_Type', 'Appl_No', 'Product_No', 'Patent_No', 'Expiration_Date',
       'Patent_Use_Code', 'Delist_Flag', 'Submission_Date',
       'Patent_Duration_Years', 'Patent_Use'],
      dtype='object')

In [116]:
order = ['Appl_No', 'Appl_Type', 'Product_No', 'Patent_No','Patent_Use_Code', 'Patent_Use','Delist_Flag', 'Submission_Date', 'Expiration_Date', 'Patent_Duration_Years'] #reorder columns to make it more readable.
df_pat = df_pat[order] #reorder columns to make it more readable.
df_pat.head()

,Appl_No,Appl_Type,Product_No,Patent_No,Patent_Use_Code,Patent_Use,Delist_Flag,Submission_Date,Expiration_Date,Patent_Duration_Years
0,20610,Innovator,1,7625884,U-141,TREATMENT OF ULCERATIVE COLITIS,Not Requested,NaT,2026-08-24,NaN
1,18613,Innovator,1,7560445,U-986,TREATMENT OF PATIENTS INFECTED WITH PEDICULUS ...,Not Requested,NaT,2027-02-01,NaN
2,19734,Innovator,2,8455524,U-1029,METHOD FOR TREATING ACUTE ELEVATIONS OF BLOOD ...,Not Requested,2013-06-27,2027-04-18,13.8
3,19734,Innovator,5,8455524,U-1029,METHOD FOR TREATING ACUTE ELEVATIONS OF BLOOD ...,Not Requested,NaT,2027-04-18,NaN
4,20832,Innovator,7,7422388,U-1397,USE AS AN ANTISEPTIC FOR THE PREPARATION OF A ...,Not Requested,NaT,2027-04-25,NaN


In [117]:
#Check for duplicated rows.

print(f"There are {df_pat.duplicated().sum()} duplicates in this dataframe.")

There are 0 duplicates in this dataframe.


In [118]:
df_pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Appl_No                20052 non-null  object        
 1   Appl_Type              20052 non-null  object        
 2   Product_No             20052 non-null  object        
 3   Patent_No              20052 non-null  object        
 4   Patent_Use_Code        20052 non-null  object        
 5   Patent_Use             20052 non-null  object        
 6   Delist_Flag            20052 non-null  object        
 7   Submission_Date        17044 non-null  datetime64[ns]
 8   Expiration_Date        20052 non-null  datetime64[ns]
 9   Patent_Duration_Years  17044 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 1.5+ MB


In [119]:
df_pat.to_excel('../1_Data/Data_Transformed/patent_tr.xlsx', index = False)